In [1]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn import svm
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
model_name = 'SVM'

mlq.start_quest(model_name, table_dest="../../")

# <font color="aqua">SVM</font> Model

### Read the data

In [2]:
x_data_d, y_data_d = mlq.l(read_data)(kind="Numerical")
opt_params = load_hyperparameters(model_name)

### Initiate model

In [3]:
#clf = mlq.l(svm.SVC)(**opt_params)
clf = mlq.l(svm.SVC)(kernel='linear', C=53)

### Evaluate Model Bias

In [4]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.980     0.987     0.984       152
           1      0.969     0.969     0.969       160
           2      0.994     0.985     0.989       324
           3      0.996     1.000     0.998       544

    accuracy                          0.990      1180
   macro avg      0.985     0.985     0.985      1180
weighted avg      0.990     0.990     0.990      1180



### Evaluate Model Generalization

In [5]:
loo, kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1)

loo_acc=loo.get('loo_acc')
loo_wf1=loo.get('loo_wf1')
repeated_10fold_acc, repeated_10fold_wf1 = kfold.get(f'10-Repeated 10-fold')[:2]    

# loo_report=loo.get('loo_report')
# repeated_10fold_report= kfold.get(f'10-Repeated 10-fold')[2]
# print(f'Leave-One-Out Report:\n {loo_report}')
# print(f'10-Repeated 10-fold Report:\n {repeated_10fold_report}')

loo_acc,10-Repeated 10-fold
0.9771,0.9839


loo_wf1,10-Repeated 10-fold
0.977,0.9839


#### Save Model

In [6]:
save_model(model_name, clf)

### Tracking

In [7]:
mlq.log_metrics(train_acc, train_wf1,loo_acc, loo_wf1, repeated_10fold_acc, repeated_10fold_wf1)
mlq.end_quest()
mlq.show_logs("SVM", table_dest="../../", last_k=6)